In [2]:
import json
import io
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
cc_json = ''


In [3]:
with open('./Pharmacy_call.json', 'r') as cc_file:
    cc_json = json.loads(cc_file.read())

In [4]:
conversation = []
i=1

for phrase in cc_json['transcription']['recognizedPhrases'] :
    if i%2:
        conversation.append({"text":phrase['nBest'][0]['display'], 'id':i, 'role':'Agent', 'participantId':'Agent_{}'.format(1)})
    else:
        conversation.append({"text":phrase['nBest'][0]['display'], 'id':i, 'role':'Customer', 'participantId':'Customer_{}'.format(2)})
    i=i+1


In [5]:
json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  'tasks': [{'taskName': 'analyze 1',
   'kind': 'ConversationalSummarizationTask',
   'parameters': {'summaryAspects': ['chaptertitle',
     'issue',
     'resolution',
     'narrative'],}}]}



In [6]:
print(json_d)

{'displayName': 'Conversation Task Example', 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hi, thank you for calling Contoso pharmacy. Who am I speaking with today?', 'id': 1, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Good afternoon. My name is Mary. I'm calling about a refill for my prescribed medications. I have been trying to get ahold of someone for weeks and was told that I would get a call back regarding my situation, but it's been weeks and no one's contacted me, so I thought I'd call.", 'id': 2, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "I understand your frustration, Mary. Umm, can you tell me what exactly you're trying to accomplish?", 'id': 3, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Yes, I'm trying to get a refill of my prescription drugs that the my doctor prescribed to me for cholesterol.", 'id': 4, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': 'I see here that you were on a generic form

In [6]:
import requests
from requests.structures import CaseInsensitiveDict

parser=ConfigParser()
_=parser.read('../config.cfg')
cog_svc_ep = parser.get('openai_api','cog_svc_ep')

url = "{}/language/analyze-conversations/jobs?api-version=2022-10-01-preview".format(cog_svc_ep)

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.post(url, headers=headers, json=data)

print(resp)


<Response [202]>


In [7]:
url = resp.headers['operation-location']

In [9]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.get(url, headers=headers)

out = resp.json()
#print(out)

In [10]:
for task in out['tasks']['items'][0]['results']['conversations'][0]['summaries']:
    print("{} = {}".format(task['aspect'],task['text']))


chapterTitle = Prescription Drugs
issue = Customer wants to get a refill for their prescription. | Customer is not sure how to get a refill.
resolution = Asked customer what exactly customer is trying to accomplish. | Explained to the customer that the doctor stopped prescribe the drug. | Helped customer to fill the prescription. | Informed customer that the prescription is now for a different drug.
narrative = Agent_1 is trying to get ahold of Mary about a refill for her prescription drugs. Agent_1 is checking the records and finds that she is now on a different drug, metformin.


In [11]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [13]:
#print("Provide detailed call summary notes of the following text:\n'''{}'''\n{}".format(conversation,'Result:'))

In [16]:
prompt = "Generate detailed call summary notes of the following text:\n'''{}'''\n{}".format(conversation,'Result:\n-')

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.2,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print('-'+response['choices'][0]['text'])

Response:
-The customer, Mary, is trying to refill a prescription for cholesterol medication that she has been trying to get ahold of someone for weeks for. 
-The agent looks up Mary's information and sees that her doctor stopped prescribing the medication in 2021. 
-The agent tells Mary that her doctor had the drugs prescribed to her from 2012 through 2021 and that she is now on a different drug, metformin. 
-The agent offers to fill the prescription for Mary for pick up. 
-Mary thanks the agent and the call ends.


In [15]:
prompt = "Provide list of medications, dose, and form discussed in the following text:\n'''{}'''\n{}".format(conversation,'Result:\n-')

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print('-'+response['choices'][0]['text'])

Response:
- Lipitor (atorvastatin)
- Metformin
